In [1]:
import logging

from external.client import YandexWeatherAPI
from tasks import (
    DataFetchingTask,
    DataCalculationTask,
    DataAggregationTask,
    DataAnalyzingTask,
)

TIMEOUT = 5.

logging.basicConfig(format='%(asctime)s [%(levelname)s] %(message)s')

In [2]:
# DataFetchingTask.
data_fetcher = DataFetchingTask(client=YandexWeatherAPI())
data_fetcher.run(timeout=TIMEOUT)

2023-08-14 11:36:01,105 [WARNING] Unable to fetch data for GIZA: HTTP Error 404: Not Found
2023-08-14 11:36:01,254 [WARNING] Unable to fetch data for TORONTO: Extra data: line 19 column 4 (char 328)


In [3]:
# DataCalculationTask.
data_calculator = DataCalculationTask(
    data_fetcher.city_to_forecasting_data
)
data_calculator.run(timeout=TIMEOUT)

In [4]:
# DataAggregationTask.
data_aggregator = DataAggregationTask(
    data_calculator.city_to_analyzed_days_info
)
aggregation_results_path = data_aggregator.run(save_format='csv')
data_aggregator.days_analyze_df.head()

,city,date,hours_start,hours_end,hours_count,temp_avg,relevant_cond_hours,temp_avg_mean,relevant_cond_hours_mean,rating
0,BEIJING,2022-05-26,9,19,11,31.818,11,31.6325,9.75,3
1,BEIJING,2022-05-27,9,19,11,32.727,11,31.6325,9.75,3
2,BEIJING,2022-05-28,9,19,11,33.818,11,31.6325,9.75,3
3,BEIJING,2022-05-29,9,14,6,28.167,6,31.6325,9.75,3
4,MOSCOW,2022-05-26,9,19,11,17.727,7,13.7500,2.00,12


In [5]:
# DataAnalyzingTask.
top_index = 3
data_analysis = DataAnalyzingTask(aggregation_results_path)
resulted_cities = data_analysis.run(top_index=3)
print(f'{top_index} most relevant cities:')
for rating, (
        city_name,
        temperature,
        relevant_days,
) in enumerate(resulted_cities):
    print(
        f'{rating + 1}. {city_name} - {temperature} °C '
        f'- {round(relevant_days)} days'
    )

1. ABUDHABI - 34.27275 °C - 9 days
2. CAIRO - 33.394 °C - 11 days
3. BEIJING - 31.6325 °C - 10 days
